In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.sql.functions import col, from_json

# 1️⃣ Secrets for Event Hub
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-order-policy")
EVENTHUB_NAME  = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-order-hub")

event_hub_config = {
    'eventhubs.connectionString': f'{CONNECTION_STR};EntityPath={EVENTHUB_NAME}'
}

# 2️⃣ Define schema for JSON body
order_schema = StructType() \
    .add("OrderID", StringType()) \
    .add("CustomerID", StringType()) \
    .add("PlanID", IntegerType()) \
    .add("OrderDate", StringType()) \
    .add("Status", StringType()) \
    .add("Amount", IntegerType())

# 3️⃣ Read from Event Hub
raw_df = spark.readStream \
    .format("eventhubs") \
    .options(**event_hub_config) \
    .load()

# 4️⃣ Parse JSON body
parsed_df = raw_df \
    .withColumn("body_str", col("body").cast("string")) \
    .withColumn("data", from_json("body_str", order_schema)) \
    .select("data.*")  # Flatten the struct

# 5️⃣ Write to Bronze (Parquet or Delta) with checkpointing
bronze_path = "abfss://bronze@telcostoragelayer.dfs.core.windows.net/order/"
checkpoint_path = bronze_path + "/_checkpoint"
display(parsed_df)
query = parsed_df.writeStream \
    .format("parquet")\
    .option("checkpointLocation", checkpoint_path) \
    .option("path", bronze_path + "/data") \
    .outputMode("append") \
    .start()
